# Set Covering Problem
**A hospital ER needs to keep doctors on call, so that a qualified
individual is available to perform every medical procedure that
might be required (there is an ocial list of such procedures). For
each of several doctors available for on-call duty, the additional
salary they need to be paid, and which procedures they can
perform, is known. The goal to choose doctors so that each
procedure is covered, at a minimum cost.**
    <img src="set_covering.png">

## Model formulation
### Sets
Set of procedure P, indexed by i, where p = 1..m

Set of of doctors D, indexd by j, where j = 1..n

### Parameters
With m procedures and n available doctors.

The data can be represented as $A \in  R^{m*n} $, where
$a_{ij} = 1$ if doctor j can perform procedure i and $a_{ij} = 0$ otherwise.

Also, let $C_j$ be the additional salary that will need to
be paid to doctor j for on-call duty.
### Vairibles 
$X_j = 1$  if doctor j is on call, and 0 otherwise
### Objective
Minimize the cost $$ \sum_j^n C_j*X_j$$
### Constraints
Ensure the coverage, for every procedure i, it has to be covered by at lease one doctor.

subject to:
$$ \sum_j^n a_{ij} * X_j \ge 1, \forall i = 1..m  $$
$$X_j \in {\{0,1 \}} ,\forall j = 1..n $$


<img src="set_covering.png">

In [1]:
import pandas as pd
import numpy as np
# d = {'one' : pd.Series([10, 20, 30, 40], index =['a', 'b', 'c', 'd']), 
#       'two' : pd.Series([10, 20, 30, 40], index =['a', 'b', 'c', 'd'])} 
  
# # creates Dataframe. 
# df = pd.DataFrame(d) 



availability_df = pd.DataFrame(
    [
    [1,0,0,1,0,0],
    [1,0,0,0,1,0],
    [0,1,1,0,0,0],
    [1,0,0,0,0,1],
    [0,1,1,0,0,1],
    [0,1,0,0,0,0]
    ],
    columns = ['doc1','doc2','doc3','doc4','doc5','doc6'],
    index = ['p1', 'p2', 'p3', 'p4','p5', 'p6']
)
availability_df

,doc1,doc2,doc3,doc4,doc5,doc6
p1,1,0,0,1,0,0
p2,1,0,0,0,1,0
p3,0,1,1,0,0,0
p4,1,0,0,0,0,1
p5,0,1,1,0,0,1
p6,0,1,0,0,0,0


### Create sets for doctors and procedures


In [2]:
from pyomo.environ import *
model = ConcreteModel()
## Create sets for doctors and procedures
model.set_doctor=Set(initialize=availability_df.columns)
model.set_procedure=Set(initialize=list(availability_df.index))

### Create parameters


In [3]:
# convert the data to dictionary
availability_d = availability_df.stack().to_dict()
availability_d

{('p1', 'doc1'): 1,
 ('p1', 'doc2'): 0,
 ('p1', 'doc3'): 0,
 ('p1', 'doc4'): 1,
 ('p1', 'doc5'): 0,
 ('p1', 'doc6'): 0,
 ('p2', 'doc1'): 1,
 ('p2', 'doc2'): 0,
 ('p2', 'doc3'): 0,
 ('p2', 'doc4'): 0,
 ('p2', 'doc5'): 1,
 ('p2', 'doc6'): 0,
 ('p3', 'doc1'): 0,
 ('p3', 'doc2'): 1,
 ('p3', 'doc3'): 1,
 ('p3', 'doc4'): 0,
 ('p3', 'doc5'): 0,
 ('p3', 'doc6'): 0,
 ('p4', 'doc1'): 1,
 ('p4', 'doc2'): 0,
 ('p4', 'doc3'): 0,
 ('p4', 'doc4'): 0,
 ('p4', 'doc5'): 0,
 ('p4', 'doc6'): 1,
 ('p5', 'doc1'): 0,
 ('p5', 'doc2'): 1,
 ('p5', 'doc3'): 1,
 ('p5', 'doc4'): 0,
 ('p5', 'doc5'): 0,
 ('p5', 'doc6'): 1,
 ('p6', 'doc1'): 0,
 ('p6', 'doc2'): 1,
 ('p6', 'doc3'): 0,
 ('p6', 'doc4'): 0,
 ('p6', 'doc5'): 0,
 ('p6', 'doc6'): 0}

In [4]:
model.availability = Param(model.set_procedure, model.set_doctor,
                  initialize= availability_d)

In [5]:
# the cost 
# cost = [2500, 2800, 800, 100, 400, 1300]
cost = [2500, 2800, 1800, 1200, 1100, 900]
cost_d = dict(zip(availability_df.columns, cost))
cost_d

{'doc1': 2500,
 'doc2': 2800,
 'doc3': 1800,
 'doc4': 1200,
 'doc5': 1100,
 'doc6': 900}

In [6]:
model.cost = Param(model.set_doctor, initialize = cost_d)

In [7]:
model.pprint()

3 Set Declarations
    availability_index : Dim=0, Dimen=2, Size=36, Domain=None, Ordered=False, Bounds=None
        Virtual
    set_doctor : Dim=0, Dimen=1, Size=6, Domain=None, Ordered=False, Bounds=None
        ['doc1', 'doc2', 'doc3', 'doc4', 'doc5', 'doc6']
    set_procedure : Dim=0, Dimen=1, Size=6, Domain=None, Ordered=False, Bounds=None
        ['p1', 'p2', 'p3', 'p4', 'p5', 'p6']

2 Param Declarations
    availability : Size=36, Index=availability_index, Domain=Any, Default=None, Mutable=False
        Key            : Value
        ('p1', 'doc1') :     1
        ('p1', 'doc2') :     0
        ('p1', 'doc3') :     0
        ('p1', 'doc4') :     1
        ('p1', 'doc5') :     0
        ('p1', 'doc6') :     0
        ('p2', 'doc1') :     1
        ('p2', 'doc2') :     0
        ('p2', 'doc3') :     0
        ('p2', 'doc4') :     0
        ('p2', 'doc5') :     1
        ('p2', 'doc6') :     0
        ('p3', 'doc1') :     0
        ('p3', 'doc2') :     1
        ('p3', 'doc3') :   

### Variable
if the doctor is picked or not

In [8]:
model.x = Var(model.set_doctor, domain = Binary)

### Objective
Minimize the cost $$ \sum_j^n C_j*X_j$$

In [9]:
def obj_func(model):
    return summation(model.cost, model.x)
# simmation = sum_product

model.obj = Objective(rule=obj_func, sense =minimize)

### Constraint
$$ \sum_j^n a_{ij} * X_j \ge 1, \forall i = 1..m  $$

In [10]:
def _avail_con(model, i):
    return sum(model.availability[i,j] * model.x[j] 
               for j in model.set_doctor) >= 1
model.demand_constraint = Constraint(model.set_procedure,
    rule = _avail_con)

### Solve 

In [11]:
solver = SolverFactory('glpk') #cbc
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 5300.0, 'Upper bound': 5300.0, 'Number of objectives': 1, 'Number of constraints': 7, 'Number of variables': 7, 'Number of nonzeros': 13, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.015442609786987305}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [12]:
for d in availability_df.columns:
    print(' ', d, ':', model.x[d]())
print('total cost:', model.obj())

  doc1 : 1.0
  doc2 : 1.0
  doc3 : 0.0
  doc4 : 0.0
  doc5 : 0.0
  doc6 : 0.0
total cost: 5300.0


In [13]:
# print_solution(model)

In [ ]:
def caught_speeding(speed, is_birthday):
    if is_birthday == 'True':
        speeding = speed - 5
    else:
        speeding = speed
        
    if speeding <= 60:
        return 'No Ticket'
    elif speeding <=80 and speeding>=61:
        return 'Small Ticket'
    else:
        return 'Big Ticket'
    pass

In [8]:
caught_speeding(81, 'True')

'Small Ticket'

In [9]:
caught_speeding(81, 'False')

'Big Ticket'